<a href="https://colab.research.google.com/github/adhikari10/U-CAM/blob/main/U_CAM_Final_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
%pip install -U -q 'google-genai'

In [ ]:
%pip install -U -q google-genai gradio

In [ ]:
# Core Gemini and Colab libraries
from google import genai
from google.genai import types
from google.colab import userdata
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Gradio for UI
import gradio as gr

# Utilities for display, file handl*ing, JSON, etc.
from IPython.display import Markdown, Image as IPImage
import json
from pydantic import BaseModel, Field # For structured JSON later
from typing import Optional, List    # For type hinting later
import requests                      # For downloading files
import pathlib                       # For handling file paths
from PIL import Image as PILImage    # For image manipulation
import io                            # For handling byte streams
import time                          # For delays (e.g., waiting for file processing)
import base64                        # For potential image decoding

print("Libraries imported successfully.")

Libraries imported successfully.


In [ ]:
client = genai.Client(api_key=GOOGLE_API_KEY)

print("Client initialized.")

# Choose a model ID to use for subsequent requests
MODEL_ID = "gemini-2.0-flash" # Other models: "gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"
print(f"Using Model ID: {MODEL_ID}")

Client initialized.
Using Model ID: gemini-2.0-flash


Sample Test Call:

In [ ]:
import json
import gradio as gr
import requests
import re

def get_gemini_analysis(input_type, amount, location, time_of_day, transaction_type, account_type, email_address, email_message):
    if amount and not amount.strip().startswith("€"):
      amount = "€ " + amount.strip("€$ ").strip()
     # Convert all inputs to lowercase to handle case insensitivity
    location = location.lower() if location else ""
    time_of_day = time_of_day.lower() if time_of_day else ""
    email_address = email_address.lower() if email_address else ""
    email_message = email_message.lower() if email_message else ""

    # Validate required inputs
    if input_type == "Transaction":
        if not all([amount, location, time_of_day, transaction_type, account_type]):
            return "⚠️ Please fill out all the required fields for a Transaction analysis."

        # Add your location validation logic if needed here (e.g., Hugging Face or other logic)
        # Validate time format


    elif input_type == "Email":
        if not all([email_address, email_message]):
            return "⚠️ Please fill out both the email address and message for an Email analysis."

    # Construct the prompt...
    if input_type == "Transaction":
        prompt = f"""
You are a cybersecurity assistant. Analyze the transaction below and return a valid JSON object with:
- "risk_assessment": an object containing:
    - "transaction_amount": the transaction amount
    - "location": the location of the transaction
    - "time_of_day": time of transaction
    - "transaction_type": e.g., Credit, Debit, Wired
    - "account_type": type of account used
    - "overall_risk": one of ["High", "Medium", "Low"]
    - "risk_reason": explain *why* this transaction is considered risky or not
    - "indicators": list of specific red flags (e.g., odd hour, foreign country, large amount). If there are no clear indicators, return ["None detected"].

    - "recommendation": a short sentence with advice for the user

- "cybersecurity_tips": a list of 5 good practices for securing transactions

Only return valid JSON. No comments or markdown.

Transaction Amount: {amount}
Location: {location}
Time of Day: {time_of_day}
Transaction Type: {transaction_type}
Account Type: {account_type}
"""


    else:
        prompt = f"""
You are a cybersecurity assistant. Analyze the email below and return a valid JSON object under the key "analysis" with:

- "subject": subject of the email (or "Unknown")
- "sender": the email address of the sender
- "risk_level": one of ["High", "Medium", "Low"]
- "risk_reason": a short sentence explaining *why* this risk level was determined
- - "indicators": a list of suspicious signs (e.g., spoofed address, urgent tone). If none are found, return ["None detected"].
- "recommendation": a single-sentence action the user should take based on the risk

Only return valid JSON. No comments or markdown.

Email Address: {email_address}
Email Message: {email_message.lower()}
"""

    try:
        config = types.GenerateContentConfig(response_mime_type="application/json")
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config=config
        )

        response_json = json.loads(response.text)
        result = f"## 🧠 Analysis Result\n\n**Input Type:** {input_type}\n\n"

        if input_type == "Transaction":
            assessment = response_json.get("risk_assessment", {})
            tips = response_json.get("cybersecurity_tips", [])

            result += f"### 🔍 Risk Assessment\n"
            risk = assessment.get("overall_risk", "Unknown")
            emoji = "🔴" if risk == "High" else "🟢" if risk == "Low" else "🟡"

            result += f"- **Transaction Amount:** {assessment.get('transaction_amount')}\n"
            result += f"- **Location:** {assessment.get('location')}\n"
            result += f"- **Time of Day:** {assessment.get('time_of_day')}\n"
            result += f"- **Transaction Type:** {assessment.get('transaction_type')}\n"
            result += f"- **Account Type:** {assessment.get('account_type')}\n"
            result += f"- **🛑 Risk Level:** {emoji} **{risk}**\n"
            result += f"- **❓ Why:** {assessment.get('risk_reason')}\n"
            result += f"- **🔍 Indicators:** {', '.join(assessment.get('indicators', []))}\n"
            result += f"- **✅ Recommendation:** {assessment.get('recommendation')}\n\n"



            result += "### 🧠 Cybersecurity Tips\n<details><summary>Show Tips</summary>\n\n"
            if tips:
                for tip in tips:
                    result += f"- {tip}\n"
            else:
                result += "- No cybersecurity tips available.\n"
            result += "</details>\n"

        else:

            analysis = response_json.get("analysis", {})
            result += f"### 📧 Email Analysis\n"
            result += f"- **Subject:** {analysis.get('subject')}\n"
            result += f"- **Sender:** {analysis.get('sender')}\n"

            risk_level = analysis.get("risk_level", "Unknown")
            emoji = "🔴" if risk_level == "High" else "🟢" if risk_level == "Low" else "🟡"

            result += f"- **🛑 Risk Level:** {emoji} **{risk_level}**\n"
            result += f"- **❓ Why:** {analysis.get('risk_reason')}\n"
            result += f"- **🔍 Indicators:** {', '.join(analysis.get('indicators', []))}\n"
            result += f"- **✅ Recommendation:** {analysis.get('recommendation')}\n"

        return result

    except Exception as e:
        return f"⚠️ Could not parse Gemini's response as JSON.\n\n**Raw Response:**\n{response.text}\n\n**Error:** {e}"

# Helper function for time validation
def is_valid_time(time):
    # Check for time formats like "3:00 AM", "3 AM", "15:00", etc.
    time_pattern = r'^(0?[1-9]|1[0-2]):([0-5][0-9])\s?(AM|PM)$|^([01]?[0-9]|2[0-3]):([0-5][0-9])$'
    return bool(re.match(time_pattern, time.strip().lower()))

# --- UI Setup ---
with gr.Blocks() as demo:
    gr.Markdown("# 🛡 U-CAM: Unified Cybersecurity AI Model")

    input_type = gr.Radio(choices=["Transaction", "Email"], label="Select Input Type")

    with gr.Accordion("Transaction Details", visible=False) as transaction_fields:
        amount = gr.Textbox(label="Transaction Amount", placeholder="Enter amount in EUR")
        location = gr.Textbox(label="Location", placeholder="e.g., New York, Nigeria")
        time_of_day = gr.Textbox(label="Time of Day", placeholder="e.g., 3AM, Evening")
        transaction_type = gr.Dropdown(choices=["Credit", "Debit", "Wired"], label="Transaction Type")
        account_type = gr.Dropdown(choices=["Savings", "Checking", "Business", "Others"], label="Account Type")
        submit_btn_transaction = gr.Button("Analyze with Gemini")

    with gr.Accordion("Email Details", visible=False) as email_fields:
        email_address = gr.Textbox(label="Email Address", placeholder="Enter the email address")
        email_message = gr.Textbox(label="Email Message", placeholder="Paste the full email text here")
        submit_btn_email = gr.Button("Analyze with Gemini")

    user_selections_box = gr.Textbox(label="Your Selections", lines=5, interactive=False)
    gemini_result_box = gr.Markdown()  # Changed from gr.Textbox to gr.Markdown

    def toggle_ui(val):
        return (
            gr.update(visible=(val == "Transaction")),
            gr.update(visible=(val == "Email")),
            gr.update(value=""),
            gr.update(value="")
        )

    input_type.change(
        fn=toggle_ui,
        inputs=input_type,
        outputs=[transaction_fields, email_fields, user_selections_box, gemini_result_box]
    )

    def display_user_input(input_type, amount, location, time_of_day, transaction_type, account_type, email_address, email_message):
        if input_type == "Transaction":
            return f"""Transaction:
- Amount: {amount}
- Location: {location}
- Time of Day: {time_of_day}
- Type: {transaction_type}
- Account: {account_type}"""
        else:
            return f"""Email:
- Address: {email_address}
- Message: {email_message[:100]}..."""

    submit_btn_transaction.click(
        fn=get_gemini_analysis,
        inputs=[input_type, amount, location, time_of_day, transaction_type, account_type, email_address, email_message],
        outputs=[gemini_result_box]
    ).then(
        fn=display_user_input,
        inputs=[input_type, amount, location, time_of_day, transaction_type, account_type, email_address, email_message],
        outputs=[user_selections_box]
    )

    submit_btn_email.click(
        fn=get_gemini_analysis,
        inputs=[input_type, amount, location, time_of_day, transaction_type, account_type, email_address, email_message],
        outputs=[gemini_result_box]
    ).then(
        fn=display_user_input,
        inputs=[input_type, amount, location, time_of_day, transaction_type, account_type, email_address, email_message],
        outputs=[user_selections_box]
    )

    gr.Markdown("""
    **Note:** This model is using AI to analyze inputs and provide real-time insights.

    ---
    ⚠️ **Disclaimer:** This tool provides general cybersecurity advice for demonstration purposes only.
    It does not replace professional threat analysis or security services.
    """)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e9bae96c86a2c3c7b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
